In [4]:
import os
import pandas as pd
import ast
import re

def find_header_row(file_path):
    required_columns = ['No', 'SEX', 'SEX(암,거세)', '등급', 'GRADE']
    
    # 엑셀 파일을 열고 각 행을 확인
    for i in range(20):
        df = pd.read_excel(file_path, header=i, nrows=1)
        if all(any(col in df.columns for col in required_columns) for col in ['No', 'SEX']):
            return i
    
    raise ValueError(f"Required columns not found in the first 20 rows of {file_path}")

# 제외할 이미지 목록 읽기 함수
def read_exclude_list(file_path):
    exclude_images = set()
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            for line in f:
                try:
                    # 문자열을 리스트로 변환
                    image_list = ast.literal_eval(line.strip())
                    # 리스트의 각 이미지를 set에 추가
                    exclude_images.update(image_list)
                except (ValueError, SyntaxError):
                    # 라인을 파싱할 수 없는 경우 무시
                    continue
    return exclude_images

def get_image_paths(image_directory):
    image_paths = []
    for root, _, files in os.walk(image_directory):
        for file in files:
            if re.match(r'[\w\+]+_\d+\.jpg', file, re.IGNORECASE) or re.match(r'\d+_\(\d+\)\.jpg', file, re.IGNORECASE):
                image_paths.append(os.path.join(root, file))
    return image_paths

def extract_image_number(file_name):
    match = re.search(r'_(\d+)\.jpg', file_name, re.IGNORECASE) or re.search(r'_(\(\d+\))\.jpg', file_name, re.IGNORECASE)
    if match:
        return int(match.group(1).strip('()'))
    return None

def process_data(base_directory, image_directories, excel_files, output_file):
    exclude_file_path1 = '차이_4이상.txt'
    exclude_file_path2 = 'nan_값_포함.txt'
    exclude_images = read_exclude_list(exclude_file_path1).union(read_exclude_list(exclude_file_path2))
    
    print(len(exclude_images))
    if exclude_images:
        print(next(iter(exclude_images)))

    dataframes = []
    current_number = 1  # 시작 번호

    for folder_name, file_path in excel_files.items():
        try:
            header_row = find_header_row(file_path)
            df = pd.read_excel(file_path, header=header_row)
            
            # 컬럼 이름 변환
            df.columns = df.columns.str.strip()
            
            # 'No' 컬럼이 존재하지 않으면 KeyError 방지
            if 'No' not in df.columns or not any(col in df.columns for col in ['SEX', 'SEX(암,거세)']) or not any(col in df.columns for col in ['등급', 'GRADE']):
                raise ValueError(f"Required columns 'No' or 'SEX' not found in {file_path}")
            
            df = df.dropna(subset=['No', 'SEX' if 'SEX' in df.columns else 'SEX(암,거세)', '등급' if '등급' in df.columns else 'GRADE'])
            
            # 'No' 열에서 숫자만 추출하여 사용
            df['Original_No'] = df['No'].apply(lambda x: int(''.join(filter(str.isdigit, str(x)))) if pd.notna(x) else None)
            
            # 이미지 경로 가져오기
            image_paths = get_image_paths(image_directories[folder_name])
            image_dict = {extract_image_number(os.path.basename(path)): path for path in image_paths if extract_image_number(os.path.basename(path)) is not None}

            # 데이터프레임에 이미지 경로 매칭
            df['SEX'] = df['SEX'] if 'SEX' in df.columns else df['SEX(암,거세)']
            df['SEX'] = df['SEX'].replace({'거세': '거'})
            df['SEX'] = df['SEX'].replace({'거세 ': '거'})
            df['grade'] = df['등급'] if '등급' in df.columns else df['GRADE']
            df['image_path'] = df['Original_No'].apply(lambda x: image_dict.get(x, None))
            
            # 존재하는 이미지 파일만 필터링하고 제외 목록에 없는 이미지만 선택
            df = df[df['image_path'].notna()]
            df = df[df['image_path'].apply(lambda x: os.path.basename(x) not in exclude_images)]
            
            # 새로운 연속적인 번호 부여
            df['No'] = range(current_number, current_number + len(df))
            current_number += len(df)  # 다음 폴더를 위해 번호 업데이트
            
            # 유효한 파일 경로가 몇 개인지 로그 출력
            if not df.empty:
                print(f"Filtered valid image paths for {folder_name}: {len(df)} / {df['Original_No'].max()}")
            else:
                print(f"No valid image paths for {folder_name}")
            
            # 필요한 열만 선택 (No, grade, SEX, image_path)
            df['grade'] = folder_name  # 폴더 이름을 grade로 사용
            df = df[['No', 'grade', 'SEX', 'image_path']]

            dataframes.append(df)

        except ValueError as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue

    # 모든 데이터프레임 병합
    if dataframes:
        all_data = pd.concat(dataframes, ignore_index=True)
        all_data.columns = all_data.columns.str.replace(r'\(.*\)', '', regex=True).str.strip()
        all_data.columns = all_data.columns.str.replace(' ', '_')
        
        # image_path에서 '../' 제거
        all_data['image_path'] = all_data['image_path'].str.replace(r'^\.\./', '', regex=True)

        # CSV 파일로 저장
        all_data.to_csv(os.path.join('../dataset/', output_file), index=False)
        print(f"Data saved to {os.path.join(base_directory, output_file)}")
    else:
        print("No data to save.")

    return dataframes if dataframes else None

base_directory = "../dataset/meat_dataset/"
label_directory = "../dataset/meat_dataset/labels"
output_file = "only_new_1112.csv"

# 이미지 파일 경로 설정 (임의의 매칭을 위해 folder_name 대신 다른 키 사용 가능)
image_directories = {
    '20240904': os.path.join(base_directory, '20240904/240904_개체사진'),
    '20240905': os.path.join(base_directory, '20240905/240905_개체사진'),
    '20240906': os.path.join(base_directory, '20240906/240906_개체사진'),
    '20240910': os.path.join(base_directory, '20240910/240910_개체사진'),
    '20240912': os.path.join(base_directory, '20240912/240912_개체사진'),
    '20240924': os.path.join(base_directory, '20240924/240924_개체사진'),
    '20240926': os.path.join(base_directory, '20240926/240926_개체사진'),
    '20240927': os.path.join(base_directory, '20240927/240927_개체사진'),
    '20241008': os.path.join(base_directory, '20241008/241008_개체사진'),
    '20241011': os.path.join(base_directory, '20241011/241011_개체사진'),
    '20241015': os.path.join(base_directory, '20241015/241015_개체사진'),
    '20241016': os.path.join(base_directory, '20241016/241016_개체사진'),
    '20241017': os.path.join(base_directory, '20241017/241017_개체사진'),
    '20241018': os.path.join(base_directory, '20241018/241018_개체사진'),
    '20241022': os.path.join(base_directory, '20241022/241022_개체사진'),
    '20241023': os.path.join(base_directory, '20241023/241023_개체사진'),
    '20241024': os.path.join(base_directory, '20241024/241024_개체사진'),
    '20241029': os.path.join(base_directory, '20241029/241029_개체사진'),
    '20241030': os.path.join(base_directory, '20241030/241030_개체사진'),
    '20241031': os.path.join(base_directory, '20241031/241031_개체사진'),
    '20241101': os.path.join(base_directory, '20241101/241101_개체사진'),
    '20241105': os.path.join(base_directory, '20241105/241105_개체사진'),
    '20241106': os.path.join(base_directory, '20241106/241106_개체사진'),
    '20241107': os.path.join(base_directory, '20241107/241107_개체사진'),
    '20241108': os.path.join(base_directory, '20241108/241108_개체사진'),
    '20241112': os.path.join(base_directory, '20241112/241112_개체사진')
}

# 엑셀 파일 경로 설정
excel_files = {
    '20240904': os.path.join(label_directory, '횡성KC_실촬영본_240904.xlsx'),
    '20240905': os.path.join(label_directory, '횡성KC_실촬영본_240905.xlsx'),
    '20240906': os.path.join(label_directory, '횡성KC_실촬영본_240906.xlsx'),
    '20240910': os.path.join(label_directory, '횡성KC_실촬영본_240910.xlsx'),
    '20240912': os.path.join(label_directory, '횡성KC_실촬영본_240912.xlsx'),
    '20240924': os.path.join(label_directory, '횡성KC_실촬영본_240924.xlsx'),
    '20240926': os.path.join(label_directory, '횡성KC_실촬영본_240926.xlsx'),
    '20240927': os.path.join(label_directory, '횡성KC_실촬영본_240927.xlsx'),
    '20241008': os.path.join(label_directory, '횡성KC_실촬영본_241008.xlsx'),
    '20241011': os.path.join(label_directory, '횡성KC_실촬영본_241011.xlsx'),
    '20241015': os.path.join(label_directory, '횡성KC_실촬영본_241015.xlsx'),
    '20241016': os.path.join(label_directory, '횡성KC_실촬영본_241016.xlsx'),
    '20241017': os.path.join(label_directory, '횡성KC_실촬영본_241017.xlsx'),
    '20241018': os.path.join(label_directory, '횡성KC_실촬영본_241018.xlsx'),
    '20241022': os.path.join(label_directory, '횡성KC_실촬영본_241022.xlsx'),
    '20241023': os.path.join(label_directory, '횡성KC_실촬영본_241023.xlsx'),
    '20241024': os.path.join(label_directory, '횡성KC_실촬영본_241024.xlsx'),
    '20241029': os.path.join(label_directory, '횡성KC_실촬영본_241029.xlsx'),
    '20241030': os.path.join(label_directory, '횡성KC_실촬영본_241030.xlsx'),
    '20241031': os.path.join(label_directory, '횡성KC_실촬영본_241031.xlsx'),
    '20241101': os.path.join(label_directory, '횡성KC_실촬영본_241101.xlsx'),
    '20241105': os.path.join(label_directory, '횡성KC_실촬영본_241105.xlsx'),
    '20241106': os.path.join(label_directory, '횡성KC_실촬영본_241106.xlsx'),
    '20241107': os.path.join(label_directory, '횡성KC_실촬영본_241107.xlsx'),
    '20241108': os.path.join(label_directory, '횡성KC_실촬영본_241108.xlsx'),
    '20241112': os.path.join(label_directory, '횡성KC_실촬영본_241112.xlsx')
}

processed_data = process_data(base_directory, image_directories, excel_files, output_file)


1604
등심1++_000471.jpg
Filtered valid image paths for 20240904: 97 / 107
Filtered valid image paths for 20240905: 101 / 101
Filtered valid image paths for 20240906: 69 / 70
Filtered valid image paths for 20240910: 11 / 110
Filtered valid image paths for 20240912: 107 / 107
Filtered valid image paths for 20240924: 119 / 120
Filtered valid image paths for 20240926: 78 / 80
Filtered valid image paths for 20240927: 90 / 91
Filtered valid image paths for 20241008: 113 / 113
Filtered valid image paths for 20241011: 60 / 60
Filtered valid image paths for 20241015: 65 / 65
Filtered valid image paths for 20241016: 65 / 65
Filtered valid image paths for 20241017: 67 / 67
Filtered valid image paths for 20241018: 44 / 44
Filtered valid image paths for 20241022: 88 / 88
Filtered valid image paths for 20241023: 47 / 47
Filtered valid image paths for 20241024: 47 / 47
Filtered valid image paths for 20241029: 111 / 111
Filtered valid image paths for 20241030: 43 / 44
Filtered valid image paths for 2024